<a href="https://colab.research.google.com/github/aiaidada/MHealth/blob/main/Posture_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, upload your files on Colab, and then Unzip it using the code below



In [4]:
!unzip images.zip -d dataset_seven
!unzip images_all.zip -d dataset_seven_all

Archive:  images.zip
   creating: dataset_seven/images/
   creating: dataset_seven/images/10_bad/
  inflating: dataset_seven/images/10_bad/bad (1).jpg  
  inflating: dataset_seven/images/10_bad/bad (10).jpg  
  inflating: dataset_seven/images/10_bad/bad (2).jpg  
  inflating: dataset_seven/images/10_bad/bad (3).jpg  
  inflating: dataset_seven/images/10_bad/bad (4).jpg  
  inflating: dataset_seven/images/10_bad/bad (5).jpg  
  inflating: dataset_seven/images/10_bad/bad (6).jpg  
  inflating: dataset_seven/images/10_bad/bad (7).jpg  
  inflating: dataset_seven/images/10_bad/bad (8).jpg  
  inflating: dataset_seven/images/10_bad/bad (9).jpg  
   creating: dataset_seven/images/10_good/
  inflating: dataset_seven/images/10_good/bad (30).jpg  
  inflating: dataset_seven/images/10_good/good (1).jpg  
  inflating: dataset_seven/images/10_good/good (2).jpg  
  inflating: dataset_seven/images/10_good/good (3).jpg  
  inflating: dataset_seven/images/10_good/good (4).jpg  
  inflating: dataset_se

Now, we will fix the data, this is only for our specific fileing system

In [5]:
import os
import shutil
from torchvision import datasets
from torchvision.transforms import ToTensor
from PIL import Image

# Define the path to the main folder containing the subfolders
main_folder_path = "/content/dataset_seven/images"

# Define the root folder for the datasets
root_dataset_folder = "/content/dataset"

# Iterate through folders 1 to 7
for x in range(1, 11):
    # Define folder names for good and bad datasets
    good_folder_name = f"{x}_good"
    bad_folder_name = f"{x}_bad"

    # Create dataset folders
    dataset_folder_path = os.path.join(root_dataset_folder, f"dataset_{x}")
    os.makedirs(dataset_folder_path, exist_ok=True)

    # Process good files with label 0
    good_folder_path = os.path.join(main_folder_path, good_folder_name)
    good_dataset_path = os.path.join(dataset_folder_path, "good")
    os.makedirs(good_dataset_path, exist_ok=True)
    for file_name in os.listdir(good_folder_path):
        file_path = os.path.join(good_folder_path, file_name)
        shutil.copy(file_path, os.path.join(good_dataset_path, file_name))

    # Process bad files with label 1
    bad_folder_path = os.path.join(main_folder_path, bad_folder_name)
    bad_dataset_path = os.path.join(dataset_folder_path, "bad")
    os.makedirs(bad_dataset_path, exist_ok=True)
    for file_name in os.listdir(bad_folder_path):
        file_path = os.path.join(bad_folder_path, file_name)
        shutil.copy(file_path, os.path.join(bad_dataset_path, file_name))


print("Datasets created successfully.")

Datasets created successfully.


In [6]:
import os
import shutil
from torchvision import datasets
from torchvision.transforms import ToTensor
from PIL import Image

# Define the path to the main folder containing the subfolders
main_folder_path = "/content/dataset_seven_all/images"

# Define the root folder for the datasets
root_dataset_folder = "/content/dataset_all"

# Iterate through folders 1 to 7
for x in range(1, 11):
    # Define folder names for good and bad datasets
    good_folder_name = f"{x}_good"
    bad_folder_name = f"{x}_bad"

    # Create dataset folders
    dataset_folder_path = os.path.join(root_dataset_folder, f"dataset_{x}")
    os.makedirs(dataset_folder_path, exist_ok=True)

    # Process good files with label 0
    good_folder_path = os.path.join(main_folder_path, good_folder_name)
    good_dataset_path = os.path.join(dataset_folder_path, "good")
    os.makedirs(good_dataset_path, exist_ok=True)
    for file_name in os.listdir(good_folder_path):
        file_path = os.path.join(good_folder_path, file_name)
        shutil.copy(file_path, os.path.join(good_dataset_path, file_name))

    # Process bad files with label 1
    bad_folder_path = os.path.join(main_folder_path, bad_folder_name)
    bad_dataset_path = os.path.join(dataset_folder_path, "bad")
    os.makedirs(bad_dataset_path, exist_ok=True)
    for file_name in os.listdir(bad_folder_path):
        file_path = os.path.join(bad_folder_path, file_name)
        shutil.copy(file_path, os.path.join(bad_dataset_path, file_name))


print("Datasets created successfully.")

Datasets created successfully.


Creating the dataset and adding the labels

In [7]:
import os
from PIL import Image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['good', 'bad']
        self.file_paths, self.labels = self._load_file_paths_and_labels()

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def _load_file_paths_and_labels(self):
        file_paths = []
        labels = []

        for class_name in self.classes:
            class_path = os.path.join(self.root_dir, class_name)
            if os.path.exists(class_path):
                class_label = 0 if class_name == 'good' else 1
                for filename in os.listdir(class_path):
                    if filename.endswith(('.jpg', '.jpeg', '.png')):
                        file_paths.append(os.path.join(class_path, filename))
                        labels.append(class_label)

        return file_paths, labels


Training with RGB images of size

In [8]:
import torch.optim as optim
import time

def train(epochs , net , train_loader ):

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

  start_time = time.time()

  for epoch in range(epochs):  # loop over the dataset multiple times

    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs.unsqueeze(0), labels)
        loss.backward()
        optimizer.step()

  end_time = time.time()
  training_time = end_time - start_time
  print(f'Training Time: {training_time:.2f} seconds')
  print('Finished Training')
  return training_time

In [9]:
def evaluate_model(model, dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images, labels
            outputs = model(images)
            total_correct = total_correct+1 if outputs.argmax() == labels else total_correct
            total_samples +=1

    accuracy = total_correct / total_samples
    return accuracy

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(6032, 2500)
        self.fc2 = nn.Linear(2500, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



transforms = transforms.Compose([transforms.Resize((128, 64)), transforms.ToTensor()])
train_times = [0]*10
train_acc= [0]*10
test_acc = [0]*10

for i in range(1 , 11):
  net = Net()
  dataset = CustomDataset(root_dir= f'/content/dataset/dataset_{i}', transform= transforms)
  for j in range(10):

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False)

    time_train = train(epochs=50 , net= net, train_loader= train_loader )
    train_times[i-1] += time_train
    train_accuracy = evaluate_model(net, train_loader)
    print(f'Training Accuracy: {train_accuracy:.4f}' , i)
    train_acc[i-1]+= train_accuracy
    # Evaluate the model on the test set
    test_accuracy = evaluate_model(net, val_loader)
    print(f'Test Accuracy: {test_accuracy:.4f}')
    test_acc[i-1]+= test_accuracy



Training Time: 152.71 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 0.8000
Training Time: 151.55 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.47 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.65 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.12 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 152.98 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 152.03 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 151.64 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.21 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 152.42 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 126.3

Some people had regestared more than 10 photos, thus, we first deleted random photo from each person so that the comparision is fair for everyone.

Also, since our dataset is quite small

In [11]:
print(train_times)
print(train_acc)
print(test_acc)

[1526.785171508789, 1264.4718692302704, 1196.7991392612457, 1113.7084741592407, 1289.7367677688599, 1285.0790226459503, 1292.0624506473541, 1010.5748569965363, 1050.4389324188232, 1041.4965760707855]
[10.0, 10.0, 10.0, 9.846153846153847, 10.0, 10.0, 9.6, 10.0, 10.0, 10.0]
[9.8, 9.25, 10.0, 8.5, 9.5, 9.5, 9.0, 10.0, 10.0, 10.0]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(6032, 2500)
        self.fc2 = nn.Linear(2500, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



transforms = transforms.Compose([transforms.Resize((128, 64)), transforms.ToTensor()])
train_times = [0]*10
train_acc= [0]*10
test_acc = [0]*10

for i in range(1 , 11):
  net = Net()
  dataset = CustomDataset(root_dir= f'/content/dataset_all/dataset_{i}', transform= transforms)
  for j in range(10):

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False)

    time_train = train(epochs=50 , net= net, train_loader= train_loader )
    train_times[i-1] += time_train
    train_accuracy = evaluate_model(net, train_loader)
    print(f'Training Accuracy: {train_accuracy:.4f}' , i)
    train_acc[i-1]+= train_accuracy
    # Evaluate the model on the test set
    test_accuracy = evaluate_model(net, val_loader)
    print(f'Test Accuracy: {test_accuracy:.4f}')
    test_acc[i-1]+= test_accuracy



Training Time: 153.41 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 0.8000
Training Time: 160.98 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 0.8000
Training Time: 167.09 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.82 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.91 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.06 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.61 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.64 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.25 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.33 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 135.8

In [ ]:
print(train_times)
print(train_acc)
print(test_acc)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(6032, 2500)
        self.fc2 = nn.Linear(2500, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



transforms = transforms.Compose([transforms.Resize((128, 64)), transforms.ToTensor()])
train_times = [0]*10
train_acc= [0]*10
test_acc = [0]*10

for i in range(1 , 11):
  net = Net()
  dataset = CustomDataset(root_dir= f'/content/dataset_all/dataset_{i}', transform= transforms)
  for j in range(10):

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False)

    time_train = train(epochs=50 , net= net, train_loader= train_loader )
    train_times[i-1] += time_train
    train_accuracy = evaluate_model(net, train_loader)
    print(f'Training Accuracy: {train_accuracy:.4f}' , i)
    train_acc[i-1]+= train_accuracy
    # Evaluate the model on the test set
    test_accuracy = evaluate_model(net, val_loader)
    print(f'Test Accuracy: {test_accuracy:.4f}')
    test_acc[i-1]+= test_accuracy



Training Time: 153.41 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 0.8000
Training Time: 160.98 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 0.8000
Training Time: 167.09 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.82 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.91 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.06 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.61 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.64 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 153.25 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 154.33 seconds
Finished Training
Training Accuracy: 1.0000 1
Test Accuracy: 1.0000
Training Time: 135.8